<a href="https://colab.research.google.com/github/von-ai/modeling-sentiment-analysis/blob/main/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(os.listdir())

['.config', 'drive', 'sample_data']


In [ ]:
os.chdir('/content/drive/MyDrive/IMDB Movie Reviews Sentiment Analysis')

In [ ]:
print(os.listdir())

['Data', 'Models', 'Data_Cleaning.ipynb', 'X_train_tfidf.pkl', 'X_test_tfidf.pkl', 'y_train.csv', 'y_test.csv', 'tfidf_vectorizer.pkl', 'final_IMDB_dataset.csv', 'Feature_Extraction.ipynb']


In [ ]:
import os
import pickle
import joblib
import numpy as np
import pandas as pd
from google.colab import drive

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#buka drive
drive.mount('/content/drive', force_remount=True)

base_path = '/content/drive/MyDrive/IMDB Movie Reviews Sentiment Analysis'
if not os.path.exists(base_path):
    raise FileNotFoundError(f"Folder tidak ada: {base_path}")

os.chdir(base_path)
print(f"📂 Current directory: {os.getcwd()}")

def safe_load(path):
    """Load pickle/joblib/numpy safely"""
    if not os.path.exists(path):
        raise FileNotFoundError(f"⚠️ File not found: {path}")

    ext = os.path.splitext(path)[1].lower()
    try:
        if ext in ['.pkl', '.pickle']:
            return joblib.load(path)
        elif ext == '.joblib':
            return joblib.load(path)
        elif ext in ['.npy', '.npz']:
            return np.load(path, allow_pickle=False)
        else:
            raise ValueError("Unknown file type")
    except Exception as e:
        raise RuntimeError(f"Failed to load {path}: {e}")

#buka dataset
print("\n🔄 Loading TF-IDF feature matrices...")
X_train = safe_load('X_train_tfidf.pkl')
X_test = safe_load('X_test_tfidf.pkl')

print("\n🔄 Loading labels...")
y_train = pd.read_csv('y_train.csv').squeeze()
y_test = pd.read_csv('y_test.csv').squeeze()


#label encoding
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)  # positive → 1, negative → 0
y_test_enc = le.transform(y_test)

print("✅ Label classes:", le.classes_)

#training model
models = {
    "Logistic Regression": LogisticRegression(max_iter=2000, class_weight='balanced'),
    "Support Vector Machine": LinearSVC(class_weight='balanced'),
    "Naive Bayes": MultinomialNB()
}

results = {}

print("\n🚀 Training models...\n")
for name, model in models.items():
    print(f"🔹 Training {name}...")
    try:
        model.fit(X_train, y_train_enc)
        preds = model.predict(X_test)
        acc = accuracy_score(y_test_enc, preds)
        results[name] = acc

        print(f"✅ {name} Accuracy: {acc:.4f}")
        print(classification_report(y_test_enc, preds, target_names=le.classes_))
        print("Confusion Matrix:\n", confusion_matrix(y_test_enc, preds))
        print("-" * 60)
    except Exception as e:
        print(f"❌ Error training {name}: {e}")


#summary
print("\n🏁 MODEL PERFORMANCE SUMMARY")
for name, acc in results.items():
    print(f"{name:<25}: {acc:.4f}")

if results:
    best_model = max(results, key=results.get)
    print(f"\n🌟 Best model: {best_model} with accuracy {results[best_model]:.4f}")
else:
    print("⚠️ No successful model training detected.")


Mounted at /content/drive
📂 Current directory: /content/drive/.shortcut-targets-by-id/1ZxE3MvSNrG1FcVVUeYVW4Y85XcoYeWPd/IMDB Movie Reviews Sentiment Analysis

🔄 Loading TF-IDF feature matrices...

🔄 Loading labels...
✅ Label classes: ['negative' 'positive']

🚀 Training models...

🔹 Training Logistic Regression...
✅ Logistic Regression Accuracy: 0.8918
              precision    recall  f1-score   support

    negative       0.90      0.88      0.89      4961
    positive       0.88      0.91      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

Confusion Matrix:
 [[4347  614]
 [ 468 4571]]
------------------------------------------------------------
🔹 Training Support Vector Machine...
✅ Support Vector Machine Accuracy: 0.8948
              precision    recall  f1-score   support

    negative       0.90      0.88      0.89      4961
    positive       0.89 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

print("\n🔄 Loading TF-IDF vectorizer...")
try:
    vectorizer = joblib.load('tfidf_vectorizer.pkl')
    print("✅ TF-IDF vectorizer loaded successfully!")
except:
    print("⚠️ Vectorizer file not found. Pastikan file 'tfidf_vectorizer.pkl' tersedia di folder yang sama.")
    raise

best_model_name = "Support Vector Machine"
model = models[best_model_name]

print(f"\n🌟 Using best model: {best_model_name}")

def predict_sentiment(text):
    """Prediksi sentimen dari teks input"""
    X_input = vectorizer.transform([text])
    pred = model.predict(X_input)[0]
    sentiment = "😊 Positive" if pred == 1 else "😞 Negative"
    return sentiment

while True:
    review = input("\n🎬 Masukkan review film (atau ketik 'exit' untuk keluar):\n> ")
    if review.lower() == 'exit':
        print("👋 Bye-bye~")
        break

    result = predict_sentiment(review)
    print(f"🔍 Hasil prediksi: {result}")



🔄 Loading TF-IDF vectorizer...
✅ TF-IDF vectorizer loaded successfully!

🌟 Using best model: Support Vector Machine

🎬 Masukkan review film (atau ketik 'exit' untuk keluar):
> good
🔍 Hasil prediksi: 😊 Positive

🎬 Masukkan review film (atau ketik 'exit' untuk keluar):
> nice film
🔍 Hasil prediksi: 😊 Positive

🎬 Masukkan review film (atau ketik 'exit' untuk keluar):
> bad film
🔍 Hasil prediksi: 😞 Negative

🎬 Masukkan review film (atau ketik 'exit' untuk keluar):
> disgusting
🔍 Hasil prediksi: 😞 Negative

🎬 Masukkan review film (atau ketik 'exit' untuk keluar):
> exit
👋 Bye-bye~
